## Imports

In [2]:
from geopy.geocoders import GoogleV3
import numpy as np
import geocoder
import json
import pandas as pd
from geojson import Feature, FeatureCollection, Point
import tabula
from tabula.io import read_pdf
import re
import os

## Import data from PDF

In [3]:
# Gotta figure out how to get rid of the java error.
# SO: https://stackoverflow.com/questions/54817211/java-command-is-not-found-from-this-python-process-please-ensure-java-is-inst

dsf = tabula.read_pdf('condo.pdf', pages='all',lattice=True)

## Pandas Manipulation

In [4]:
df = dsf[0]
df.columns = df.iloc[0]
df = df.drop(df.index[0])
### This next line gets ride of the Nan ###
df = df.iloc[: , 1:]
df = df.replace(np.nan, 'Not Available', regex=True)
df = df.rename(columns={'Building / Address / City': 'building_address_city','Days on\rMarket':'days_on_market',\
                        'Price /\rSq. Ft.':'price_per_sqft'})
df['geo_address'] = df['building_address_city']
df['geo_address'] = df['geo_address'].map(lambda x: re.sub(r'\r', ' ', x))
df = df.replace('\n',' ',regex=True)
df2 = df['geo_address'].str.extract(r'(?P<building_name>\D*)(?P<address_only>\d.*)', expand=True)

Review this later (it's about how to properly merge DFs): https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

In [5]:
result = pd.merge(df,df2,left_index=True, right_index=True)
df = result

In [6]:
df

,building_address_city,List Date,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,Buyer Broker,geo_address,building_name,address_only
1,The Muse Residences\r17141 Collins Ave UPH-1\r...,11/15/2021,8/3/2022,107,"$12,500,000.00",8422,"$2,047.17",Billy Hernandez,Douglas Elliman,Mariam Kaira,Optimar International Realty,The Muse Residences 17141 Collins Ave UPH-1 Su...,The Muse Residences,17141 Collins Ave UPH-1 Sunny Isles Beach
2,One Ocean\r1 Collins Ave 507\rMiami Beach,Not Available,8/1/2022,Not Available,"$6,300,000.00",3041,"$2,071.69",Comp Sale,Comp Only Sales,Jonathan Corso,Coldwell Banker Realty,One Ocean 1 Collins Ave 507 Miami Beach,One Ocean,1 Collins Ave 507 Miami Beach
3,Ocean Tower Three Condo\r781 Crandon Blvd 1204...,5/6/2021,8/1/2022,363,"$5,000,000.00",2500,"$2,000.00",Monica Ferragut,BHHS EWM Realty,Denise Haas,Gimeza A Real Estate Corp.,Ocean Tower Three Condo 781 Crandon Blvd 1204 ...,Ocean Tower Three Condo,781 Crandon Blvd 1204 Key Biscayne
4,Ocean Tower Two\r791 Crandon Blvd 307\rKey Bis...,8/4/2021,8/1/2022,357,"$4,075,000.00",3581,"$1,137.95",Lucia Marin,One Sotheby's INT'L Realty,Antonio Araujo,"Stroomline, LLC.",Ocean Tower Two 791 Crandon Blvd 307 Key Biscayne,Ocean Tower Two,791 Crandon Blvd 307 Key Biscayne
5,The Surf Club\r9111 Collins Ave N612\rSurfside,7/20/2022,8/1/2022,7,"$3,700,000.00",1227,"$3,015.48",Ximena Penuela,"Fort Realty, LLC",Ximena Penuela,"Fort Realty, LLC",The Surf Club 9111 Collins Ave N612 Surfside,The Surf Club,9111 Collins Ave N612 Surfside
6,101 Bar Harbour\r10155 Collins Ave 710\rBal Ha...,3/22/2021,8/2/2022,415,"$2,495,000.00",3300,$756.06,Carolina Powell,Premier Brokers International,Ryan Mendell,Maxwell E Realty Inc,101 Bar Harbour 10155 Collins Ave 710 Bal Harbour,,101 Bar Harbour 10155 Collins Ave 710 Bal Harbour
7,Balmoral\r9801 Collins Ave 9Z\rBal Harbour,5/19/2022,8/5/2022,14,"$2,175,000.00",1984,"$1,096.27",Rachel Wertheimer,One Sotheby's INT'L Realty,Boris Vertsberger,"FIP Realty Services, LLC",Balmoral 9801 Collins Ave 9Z Bal Harbour,Balmoral,9801 Collins Ave 9Z Bal Harbour
8,The Pinnacle Condo\r17555 Collins Ave 1805\rSu...,4/28/2022,8/1/2022,59,"$1,800,000.00",1840,$978.26,Joseph Menachem,United Realty Group Inc,Ralph De Martino,Ocean International Realty,The Pinnacle Condo 17555 Collins Ave 1805 Sunn...,The Pinnacle Condo,17555 Collins Ave 1805 Sunny Isles Beach
9,Ocean Place\r226 Ocean Dr 8G\rMiami Beach,5/25/2022,8/1/2022,14,"$1,725,000.00",1219,"$1,415.09",George Sauvigne,Luxury Real Estate Group LLC,Jerome Brun De Saint Hippolyte,The Corcoran Group,Ocean Place 226 Ocean Dr 8G Miami Beach,Ocean Place,226 Ocean Dr 8G Miami Beach
10,T0rump Royale\r18201 Collins Ave 806\rSunny Is...,1/10/2022,8/3/2022,192,"$1,700,000.00",2137,$795.51,Oscar Donado,Kismet 2 Realty Inc.,Felipe Quintero,EXP Realty LLC,T0rump Royale 18201 Collins Ave 806 Sunny Isle...,T,0rump Royale 18201 Collins Ave 806 Sunny Isles...


In [7]:
df.columns

Index(['building_address_city', 'List Date', 'Close Date', 'days_on_market',
       'Sale Price', 'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker',
       'Buyer Agent', 'Buyer Broker', 'geo_address', 'building_name',
       'address_only'],
      dtype='object')

## Geocoder Setup and Execution

In [8]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [9]:
df['loc'] = df['address_only'].apply(geolocator.geocode, timeout=10)

In [10]:
df['loc']

1     (17141 Collins Ave, Sunny Isles Beach, FL 3316...
2     (1 Collins Ave Unit 507, Miami Beach, FL 33139...
3     (781 Crandon Blvd APT 1204, Key Biscayne, FL 3...
4     (791 Crandon Blvd APT 307, Key Biscayne, FL 33...
5     (9111 Collins Ave n612, Surfside, FL 33154, US...
6     (10155 Collins Ave APT 710, Bal Harbour, FL 33...
7     (9801 Collins Ave #9z, Bal Harbour, FL 33154, ...
8     (17555 Collins Ave APT 1805, Sunny Isles Beach...
9     (226 Ocean Dr #8g, Miami Beach, FL 33139, USA,...
10    (Trump Royale, 18201 Collins Ave #806, Sunny I...
Name: loc, dtype: object

In [11]:
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)

Columns must be same length as key explained
- https://cumsum.wordpress.com/2021/06/01/pandas-valueerror-columns-must-be-same-length-as-key/

Creating `df2` based on `df` row indicies.

https://datascience.stackexchange.com/questions/77033/creating-a-new-dataframe-with-specific-row-numbers-from-another

What's tripping this error is when the lat/lon/alititude generator is fed a Nan value to parse - it's effectively trying to split a `null` value into three separate parts - which is impossible.

The code below effectively caputres the anatomy of this error. It creates a new `df` based on a list of indicies from an existing one. Row `56` has no `point` since the `geocoder` couldn't find coordinates for it. The other three do. Splitting the `point` column fails only when row `56` - with `null` values - is attempted.
```
df2 = df.iloc[[56,57,58,59],:]
df2[['lat','lon','altitude']] = pd.DataFrame(df2['point'].to_list(),index=df2.index)
```

In [12]:
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

df

,building_address_city,List Date,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,Buyer Broker,geo_address,building_name,address_only,loc,point,lat,lon,altitude
1,The Muse Residences\r17141 Collins Ave UPH-1\r...,11/15/2021,8/3/2022,107,"$12,500,000.00",8422,"$2,047.17",Billy Hernandez,Douglas Elliman,Mariam Kaira,Optimar International Realty,The Muse Residences 17141 Collins Ave UPH-1 Su...,The Muse Residences,17141 Collins Ave UPH-1 Sunny Isles Beach,"(17141 Collins Ave, Sunny Isles Beach, FL 3316...","(25.934944, -80.12091819999999, 0.0)",25.934944,-80.120918,0.0
2,One Ocean\r1 Collins Ave 507\rMiami Beach,Not Available,8/1/2022,Not Available,"$6,300,000.00",3041,"$2,071.69",Comp Sale,Comp Only Sales,Jonathan Corso,Coldwell Banker Realty,One Ocean 1 Collins Ave 507 Miami Beach,One Ocean,1 Collins Ave 507 Miami Beach,"(1 Collins Ave Unit 507, Miami Beach, FL 33139...","(25.768079, -80.1339531, 0.0)",25.768079,-80.133953,0.0
3,Ocean Tower Three Condo\r781 Crandon Blvd 1204...,5/6/2021,8/1/2022,363,"$5,000,000.00",2500,"$2,000.00",Monica Ferragut,BHHS EWM Realty,Denise Haas,Gimeza A Real Estate Corp.,Ocean Tower Three Condo 781 Crandon Blvd 1204 ...,Ocean Tower Three Condo,781 Crandon Blvd 1204 Key Biscayne,"(781 Crandon Blvd APT 1204, Key Biscayne, FL 3...","(25.6882815, -80.1589302, 0.0)",25.688281,-80.158930,0.0
4,Ocean Tower Two\r791 Crandon Blvd 307\rKey Bis...,8/4/2021,8/1/2022,357,"$4,075,000.00",3581,"$1,137.95",Lucia Marin,One Sotheby's INT'L Realty,Antonio Araujo,"Stroomline, LLC.",Ocean Tower Two 791 Crandon Blvd 307 Key Biscayne,Ocean Tower Two,791 Crandon Blvd 307 Key Biscayne,"(791 Crandon Blvd APT 307, Key Biscayne, FL 33...","(25.6883367, -80.1579345, 0.0)",25.688337,-80.157934,0.0
5,The Surf Club\r9111 Collins Ave N612\rSurfside,7/20/2022,8/1/2022,7,"$3,700,000.00",1227,"$3,015.48",Ximena Penuela,"Fort Realty, LLC",Ximena Penuela,"Fort Realty, LLC",The Surf Club 9111 Collins Ave N612 Surfside,The Surf Club,9111 Collins Ave N612 Surfside,"(9111 Collins Ave n612, Surfside, FL 33154, US...","(25.8780812, -80.1217131, 0.0)",25.878081,-80.121713,0.0
6,101 Bar Harbour\r10155 Collins Ave 710\rBal Ha...,3/22/2021,8/2/2022,415,"$2,495,000.00",3300,$756.06,Carolina Powell,Premier Brokers International,Ryan Mendell,Maxwell E Realty Inc,101 Bar Harbour 10155 Collins Ave 710 Bal Harbour,,101 Bar Harbour 10155 Collins Ave 710 Bal Harbour,"(10155 Collins Ave APT 710, Bal Harbour, FL 33...","(25.8932332, -80.1230573, 0.0)",25.893233,-80.123057,0.0
7,Balmoral\r9801 Collins Ave 9Z\rBal Harbour,5/19/2022,8/5/2022,14,"$2,175,000.00",1984,"$1,096.27",Rachel Wertheimer,One Sotheby's INT'L Realty,Boris Vertsberger,"FIP Realty Services, LLC",Balmoral 9801 Collins Ave 9Z Bal Harbour,Balmoral,9801 Collins Ave 9Z Bal Harbour,"(9801 Collins Ave #9z, Bal Harbour, FL 33154, ...","(25.8902347, -80.1229685, 0.0)",25.890235,-80.122968,0.0
8,The Pinnacle Condo\r17555 Collins Ave 1805\rSu...,4/28/2022,8/1/2022,59,"$1,800,000.00",1840,$978.26,Joseph Menachem,United Realty Group Inc,Ralph De Martino,Ocean International Realty,The Pinnacle Condo 17555 Collins Ave 1805 Sunn...,The Pinnacle Condo,17555 Collins Ave 1805 Sunny Isles Beach,"(17555 Collins Ave APT 1805, Sunny Isles Beach...","(25.9392246, -80.12062259999999, 0.0)",25.939225,-80.120623,0.0
9,Ocean Place\r226 Ocean Dr 8G\rMiami Beach,5/25/2022,8/1/2022,14,"$1,725,000.00",1219,"$1,415.09",George Sauvigne,Luxury Real Estate Group LLC,Jerome Brun De Saint Hippolyte,The Corcoran Group,Ocean Place 226 Ocean Dr 8G Miami Beach,Ocean Place,226 Ocean Dr 8G Miami Beach,"(226 Ocean Dr #8g, Miami Beach, FL 33139, USA,...","(25.7714294, -80.1329399, 0.0)",25.771429,-80.132940,0.0
10,T0rump Royale\r18201 Collins Ave 806\rSunny Is...,1/10/2022,8/3/2022,192,"$1,700,000.00",2137,$795.51,Oscar Donado,Kismet 2 Realty Inc.,Felipe Quintero,EXP Realty LLC,T0rump Royale 18201 Collins Ave 806 Sunny Isle...,T,0rump Royale 18201 Collins Ave 806 Sunny Isles...,"(Trump Roy

Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

## Correction Section
To be used if a name is off. Format for making corrections is as follows:

`df.at[index_num,'column_name']=('what you want inserted')`

In [13]:
df.at[6,'building_name']=('101 Bal Harbour')

df.at[10,'building_name']=('Trump Royale')
df.at[10,'address_only']=('18201 Collins Ave 806')

In [14]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=11)
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format('July 31st - August 6th')

# marker_cluster = MarkerCluster().add_to(m)

for i,r in df.iterrows():
    location = (r["lat"], r["lon"])
    info = (r["Sale Price"], r["days_on_market"], r["Agent"], r['Buyer Agent'], r['price_per_sqft'], r['address_only'], r['building_name'])
    info = list(info)
    new_line = '<br>'
    bold_start = '<strong>'
    bold_end = '</strong>'
    text = f'Building Name: {bold_start}{info[-1]}{bold_end}{new_line} \
    Sale Price: {bold_start}{info[0]}{bold_end}{new_line}Days on Market: \
    {bold_start}{info[1]}{bold_end}{new_line}Listing Agent: {bold_start}{info[2]} \
    {bold_end}{new_line}Buying Agent: {bold_start}{info[3]}{bold_end}{new_line}Price sq ft: \
    {bold_start}{info[4]}{bold_end}{new_line}Address: {bold_start}{info[5]}{bold_end}'
    folium.Marker(location=location,
                      tooltip=text)\
    .add_to(m)

m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))

m

In [15]:
m.save('index.html')

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [16]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [17]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/CondoSalesWeekEnding08_06_22
